## Objective

download all genomes in an NCBI accession list table
e.g. [this table](https://www.ncbi.nlm.nih.gov/labs/virus/vssi/#/virus?SeqType_s=Nucleotide&VirusLineage_ss=SARS-CoV-2,%20taxid:2697049)

In [1]:
# papermill parameters
base="$(homedir())/workspace"
project="sars-cov2-pangenome-analysis"

accession_table_file = "$(base)/$(project)/metadata/sequences.csv"
output_directory = "$(base)/$(project)/data/sequences"

"/home/jovyan/workspace/sars-cov2-pangenome-analysis/data/sequences"

## Materials, Methods, and Functions

In [2]:
mkpath(output_directory)

"/home/jovyan/workspace/sars-cov2-pangenome-analysis/data/sequences"

In [3]:
import Pkg
Pkg.update()

pkgs = [
    "Revise",
    "DataFrames",
    "uCSV",
    "ProgressMeter",
    "FASTX",
    "Revise"
]

for pkg in pkgs
    try
        eval(Meta.parse("import $pkg"))
    catch
        Pkg.add(pkg)
        eval(Meta.parse("import $pkg"))
    end
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
  No Changes to `~/work/Mycelia/Project.toml`
  No Changes to `~/work/Mycelia/Manifest.toml`


In [4]:
# @time accession_table = DataFrames.DataFrame(uCSV.read(accession_table_file, typedetectrows=100, header=1, quotes='"')...)

In [ ]:
# sort files by reverse size
# this is helpful to track down incomplete sequences
# can also determine the size of each genome
# then determine if there is a good cutoff at which to just remove the incomplete ones
# ls -lShr

p = ProgressMeter.Progress(countlines(accession_table_file))
table_io = open(accession_table_file)
header = readline(table_io)
# ProgressMeter.@showprogress for accession in accession_table[!, "Accession"]
for line in eachline(table_io)
    accession = split(line, ',')[1]
    outfile = joinpath(output_directory, "$(accession).fna")
    if !(outfile) || (filesize(outfile) == 0)
        fasta_records = collect(Mycelia.get_sequence(db="nuccore", accession=accession))
        open(outfile, "w") do io
            fastx_io = FASTX.FASTA.Writer(io)
            for record in fasta_records
                write(fastx_io, record)
            end
            close(fastx_io)
        end
    end
    ProgressMeter.next!(p)
end
table_io

Progress:   4%|█▋                                       |  ETA: 4:05:0039m